In [4]:
# 0) Imports & settings
import os, sys, warnings, textwrap
import numpy as np
import pandas as pd

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 160)

RAW_PATH = "../data/raw/Telco-Customer-Churn.csv"
PROCESSED_DIR = "../data/processed"
REPORTS_DIR = "../reports"
os.makedirs(PROCESSED_DIR, exist_ok=True)
os.makedirs(REPORTS_DIR, exist_ok=True)

# Utility: nice printing
def title(s):
    print("\n" + "="*len(s))
    print(s)
    print("="*len(s))
print("Imports & settings done ✅")


Imports & settings done ✅


In [5]:
title("1) LOAD DATA")

# Read as strings first to avoid dtype issues, especially 'TotalCharges'
df_raw = pd.read_csv(RAW_PATH, dtype=str)

print("Shape:", df_raw.shape)
print("Columns:", list(df_raw.columns), "...")  # preview
display(df_raw.head(3))
display(df_raw.tail(3))

print("data loaded ✅")


1) LOAD DATA
Shape: (7043, 21)
Columns: ['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'] ...


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,Yes,Electronic check,29.6,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Mailed check,74.4,306.6,Yes
7042,3186-AJIEK,Male,0,No,No,66,Yes,No,Fiber optic,Yes,No,Yes,Yes,Yes,Yes,Two year,Yes,Bank transfer (automatic),105.65,6844.5,No


data loaded ✅


In [6]:
title("2) BASIC SCHEMA & DTYPE INSPECTION")

# Strip column name whitespace just in case
df = df_raw.copy()
df.columns = df.columns.str.strip()

# Trim whitespace inside string cells
for c in df.columns:
    if df[c].dtype == "object":
        df[c] = df[c].astype(str).str.strip()

# Convert target to clean binary later; for now just inspect
print(df.info())



2) BASIC SCHEMA & DTYPE INSPECTION
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   customerID        7043 non-null   object
 1   gender            7043 non-null   object
 2   SeniorCitizen     7043 non-null   object
 3   Partner           7043 non-null   object
 4   Dependents        7043 non-null   object
 5   tenure            7043 non-null   object
 6   PhoneService      7043 non-null   object
 7   MultipleLines     7043 non-null   object
 8   InternetService   7043 non-null   object
 9   OnlineSecurity    7043 non-null   object
 10  OnlineBackup      7043 non-null   object
 11  DeviceProtection  7043 non-null   object
 12  TechSupport       7043 non-null   object
 13  StreamingTV       7043 non-null   object
 14  StreamingMovies   7043 non-null   object
 15  Contract          7043 non-null   object
 16  PaperlessBilling  7043 n

In [7]:
title("3) TARGET SANITY CHECK — 'Churn'")

if "Churn" not in df.columns:
    raise ValueError("Expected a 'Churn' column. Please confirm the dataset file.")

print(df["Churn"].value_counts(dropna=False))
print("\nChurn rate (%):")
print((df["Churn"].value_counts(normalize=True) * 100).round(2))



3) TARGET SANITY CHECK — 'Churn'
Churn
No     5174
Yes    1869
Name: count, dtype: int64

Churn rate (%):
Churn
No     73.46
Yes    26.54
Name: proportion, dtype: float64


In [8]:
title("4) NUMERIC CONVERSIONS")

# Known numeric columns in Telco dataset
numeric_candidates = ["tenure", "MonthlyCharges", "TotalCharges"]

for col in numeric_candidates:
    if col in df.columns:
        # Turn blank strings into NaN then to numeric
        df[col] = df[col].replace({"": np.nan})
        df[col] = pd.to_numeric(df[col], errors="coerce")

# Show conversion result
display(df[numeric_candidates].describe(include="all"))



4) NUMERIC CONVERSIONS


,tenure,MonthlyCharges,TotalCharges
count,7043.000000,7043.000000,7032.000000
mean,32.371149,64.761692,2283.300441
std,24.559481,30.090047,2266.771362
min,0.000000,18.250000,18.800000
25%,9.000000,35.500000,401.450000
50%,29.000000,70.350000,1397.475000
75%,55.000000,89.850000,3794.737500
max,72.000000,118.750000,8684.800000


In [9]:
title("5) MISSING VALUES & DUPLICATES")

missing = (
    df.isna().sum()
    .to_frame("missing_count")
    .assign(missing_pct=lambda d: (d["missing_count"] / len(df) * 100).round(2))
    .sort_values("missing_count", ascending=False)
)
display(missing.head(20))

dup_count = df.duplicated().sum()
print(f"Duplicate rows: {dup_count}")

# Save a simple data quality report
missing.to_csv(f"{REPORTS_DIR}/missing_values_step1.csv")



5) MISSING VALUES & DUPLICATES


,missing_count,missing_pct
TotalCharges,11,0.16
customerID,0,0.00
DeviceProtection,0,0.00
MonthlyCharges,0,0.00
PaymentMethod,0,0.00
PaperlessBilling,0,0.00
Contract,0,0.00
StreamingMovies,0,0.00
StreamingTV,0,0.00
TechSupport,0,0.00


Duplicate rows: 0


In [10]:
title("6) CATEGORICAL CARDINALITY")

cat_cols = [c for c in df.columns if df[c].dtype == "object" and c != "customerID"]
cardinality = (
    pd.Series({c: df[c].nunique(dropna=False) for c in cat_cols})
    .sort_values(ascending=False)
    .to_frame("unique_values")
)
display(cardinality.head(20))
cardinality.to_csv(f"{REPORTS_DIR}/categorical_cardinality_step1.csv")



6) CATEGORICAL CARDINALITY


,unique_values
PaymentMethod,4
OnlineBackup,3
StreamingTV,3
Contract,3
MultipleLines,3
InternetService,3
OnlineSecurity,3
StreamingMovies,3
DeviceProtection,3
TechSupport,3


In [11]:
title("7) DESCRIPTIVE STATISTICS")

num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
display(df[num_cols].describe().T)

# Top levels for key categoricals (change as needed)
for c in ["Contract", "PaymentMethod", "InternetService", "gender", "Partner", "Dependents"]:
    if c in df.columns:
        print(f"\n-- {c} --")
        display(df[c].value_counts(dropna=False).head(10))



7) DESCRIPTIVE STATISTICS


,count,mean,std,min,25%,50%,75%,max
tenure,7043.0,32.371149,24.559481,0.00,9.00,29.000,55.0000,72.00
MonthlyCharges,7043.0,64.761692,30.090047,18.25,35.50,70.350,89.8500,118.75
TotalCharges,7032.0,2283.300441,2266.771362,18.80,401.45,1397.475,3794.7375,8684.80



-- Contract --


Contract
Month-to-month    3875
Two year          1695
One year          1473
Name: count, dtype: int64


-- PaymentMethod --


...

gender
Male      3555
Female    3488
Name: count, dtype: int64


-- Partner --


Partner
No     3641
Yes    3402
Name: count, dtype: int64


-- Dependents --


Dependents
No     4933
Yes    2110
Name: count, dtype: int64

In [12]:
title("8) QUICK BUSINESS SANITY CHECKS")

# Churn by Contract type
if {"Churn","Contract"} <= set(df.columns):
    churn_by_contract = (
        df.groupby("Contract")["Churn"]
          .value_counts(normalize=True)
          .rename("pct")
          .mul(100).round(2)
          .unstack(fill_value=0)
          .sort_values(by="Yes", ascending=False)
    )
    print("Churn % by Contract (rows=contract, columns=No/Yes):")
    display(churn_by_contract)

# Churn by Tenure bins
if "tenure" in df.columns:
    bins = [0, 6, 12, 24, 36, 60, df["tenure"].max()]
    labels = ["0-6","7-12","13-24","25-36","37-60","60+"]
    df["tenure_bin"] = pd.cut(df["tenure"], bins=bins, labels=labels, include_lowest=True)
    if "Churn" in df.columns:
        churn_by_tenure = (
            df.groupby("tenure_bin")["Churn"]
              .value_counts(normalize=True)
              .rename("pct")
              .mul(100).round(2)
              .unstack(fill_value=0)
        )
        print("\nChurn % by Tenure bin:")
        display(churn_by_tenure)



8) QUICK BUSINESS SANITY CHECKS
Churn % by Contract (rows=contract, columns=No/Yes):


Churn,No,Yes
Contract,,
Month-to-month,57.29,42.71
One year,88.73,11.27
Two year,97.17,2.83



Churn % by Tenure bin:


Churn,No,Yes
tenure_bin,,
0-6,47.06,52.94
7-12,64.11,35.89
13-24,71.29,28.71
25-36,78.37,21.63
37-60,83.38,16.62
60+,93.39,6.61


In [13]:
title("9) SAVE CLEANED COPY FOR STEP 2")

out_path = f"{PROCESSED_DIR}/telco_churn_step1_clean.csv"
df.to_csv(out_path, index=False)
print("Saved:", out_path)



9) SAVE CLEANED COPY FOR STEP 2
Saved: ../data/processed/telco_churn_step1_clean.csv
